패키지 다운로드

In [33]:
from dotenv import load_dotenv

load_dotenv()

True

# 1️⃣ LLM을 활용해서 답변을 생성하는 가장 기본적인 방법

In [34]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [35]:
from langchain_core.prompts import PromptTemplate

question = '인프런에는 어떤 강의가 있나요?'
prompt_with_template = f"아래 질문에 답변해주세요: \n\n {question}"

prompt_template = PromptTemplate(template=prompt_with_template, input_variables=["question"])

1. 하드 코딩

In [36]:
llm.invoke(prompt_template.invoke({"question": question}))

AIMessage(content='인프런은 다양한 주제의 온라인 강의를 제공하는 플랫폼입니다. 주로 IT, 프로그래밍, 데이터 분석, 디자인, 마케팅, 비즈니스 등 여러 분야의 강의가 있습니다. 예를 들어, Python, Java, 웹 개발, 머신러닝, UX/UI 디자인, 디지털 마케팅 등의 강의를 찾을 수 있습니다. 강의는 초급부터 고급까지 다양한 수준으로 제공되며, 실습 중심의 강의도 많아 실제 프로젝트에 적용할 수 있는 기술을 배울 수 있습니다. 특정 강의에 대한 정보는 인프런 웹사이트를 방문하여 확인할 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 134, 'prompt_tokens': 27, 'total_tokens': 161, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CZAnMdyyeZDr45yINPQ2gP8XlumoP', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--ae44f59d-28b6-45e0-a5b8-3f1b93d1b986-0', usage_metadata={'input_tokens': 27, 'output_tokens': 134, 'total_tokens': 161, 'input_token_details': {'audi

2. 체인 사용

In [37]:
prompt_chain = prompt_template | llm
prompt_chain.invoke({"question": question})

AIMessage(content='인프런은 다양한 주제의 온라인 강의를 제공하는 플랫폼입니다. 주로 IT, 프로그래밍, 데이터 분석, 디자인, 마케팅, 비즈니스 등 여러 분야의 강의가 있습니다. 예를 들어, Python, Java, 웹 개발, 머신러닝, UX/UI 디자인, 디지털 마케팅 등의 강의를 찾을 수 있습니다. 강의는 초급부터 고급까지 다양한 수준으로 제공되며, 실습 중심의 강의도 많아 실제 프로젝트에 적용할 수 있는 기술을 배울 수 있습니다. 특정 강의에 대한 정보는 인프런 웹사이트를 방문하여 확인할 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 134, 'prompt_tokens': 27, 'total_tokens': 161, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CZAnQQX7tLG2LN8VWktub7XETH40L', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--3a6cf193-8abe-4585-befa-ce6113e1f38f-0', usage_metadata={'input_tokens': 27, 'output_tokens': 134, 'total_tokens': 161, 'input_token_details': {'audi

# 2️⃣ WebBaseLoader를 사용한 RAG

In [38]:
%pip install -qU beatifulsoup4

ERROR: Could not find a version that satisfies the requirement beatifulsoup4 (from versions: none)
ERROR: No matching distribution found for beatifulsoup4
Note: you may need to restart the kernel to use updated packages.


## 1. 데이터 전처리

In [39]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.inflearn.com/courses/it-programming?completionAttributionToken=&selectedSuggestion=&selectedPosition=")

In [40]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,  # 하나의 청크가 가질 수 있는 토큰 수
    chunk_overlap=200  # 이전 청크와 겹치게 가져올 토큰 수. 유사도 검색을 할 때 원하는 문서를 가져올 수 있는 확률을 높히는 방식
)

document_list = loader.load_and_split(text_splitter=text_splitter)

In [41]:
document_list

[Document(metadata={'source': 'https://www.inflearn.com/courses/it-programming?completionAttributionToken=&selectedSuggestion=&selectedPosition=', 'title': '인프런 - 라이프타임 커리어 플랫폼', 'description': '프로그래밍, 인공지능, 데이터, 마케팅, 디자인등 입문부터 실전까지 업계 최고 선배들에게 배울 수 있는 곳.', 'language': 'ko'}, page_content='인프런 - 라이프타임 커리어 플랫폼NN인프런 - 라이프타임 커리어 플랫폼')]

In [43]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

embedding = OpenAIEmbeddings(model="text-embedding-3-large")

database = Chroma.from_documents(
    documents=document_list,
    embedding=embedding,
    collection_name="inflearn-lectures",
    persist_directory="./inflearn-lectures"
)

InternalError: Database error: error returned from database: (code: 14) unable to open database file

## 2. 데이터 검색을 위한 Retriever 생성

In [ ]:
retrieved_docs = database.as_retriever(search_kwargs={'k': 1}).invoke(question)

## 3. 프롬프트 생성

1. 수동 생성

In [ ]:
rag_prompt_template = PromptTemplate(
    template='''
    You are a helpful assistant that can answer questions about the Inflearn website.
    You are given the following context:
    {context}
    Question: {question}
    ''',
    input_variables=["context", "question"]
)

rag_chain = rag_prompt_template | llm
rag_chain.invoke({"context": retrieved_docs, "question": question})

2. LangChain Hub 제공 프롬프트 활용

In [ ]:
from langchain import hub

rlm_rag_prompt = hub.pull("rlm/rag-prompt")

rlm_rag_chain = rlm_rag_prompt | llm
rlm_rag_chain.invoke({"context": retrieved_docs, "question": question})

3. `RunnablePassThrough` 활용

In [ ]:
from langchain_core.runnables import RunnablePassthrough

rag_pass_chain = (
        {"context": retrieved_docs, "question": RunnablePassthrough()} |
        rlm_rag_prompt |
        llm
)
rag_pass_chain.invoke(question)